In [1]:
import cv2
import h5py
import winsound
import numpy as np

from time import strftime
from collections import deque
from timeit import default_timer as timer
from tensorflow.keras.models import load_model

# Define the labels of CIFAR-10 datasest
labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# Load our model that trained by 25 epochs on CIFAR dataset
model = load_model(r'C:\file location..')

# Print trained model summery
model.summary()

# Catch frame from webcam
camera = cv2.VideoCapture(1)

# This path is location for the sound file
soundPath = (r'C:\file location..'') 

# Scale to decrease the frame size
SCALE = 1

# Define variables for hight and width shape of the frames
HEIGH, WIDTH = 300, 400   

# Define variables for hight and width shape to prediction model input
hight, width = 224, 224

# Define objects boundaries size
MIN_OBJECT_AREA = 1000
MAX_OBJECT_AREA = 10000

# Define the tresh hold of the masks
DIFF_TRESH_HOLD = 20  # Should be low
MASK_TRESH_HOLD = 100 # Should be high

# Define the background
last_frame = np.zeros((int(HEIGH/SCALE), int(WIDTH/SCALE),3) , np.uint8)

# Initializing deque object for center points of the detected object
points = deque(maxlen=50)

# Define timer to check the tracking
tracking_check = 15

# Variable store the system status of tracking or not tracking
tracking_on = False

# Restart timer for FPS
fps_start = timer()    

# Define counter
counter_frames_processing = 0
counter_frames_prediction = 0
counter_frames_prediction = 0
counter_fail_predictions = 0
counter_birds_prediction = 0 
counter_frames_tracking = 0
counter_frames_reading = 0    

# Increasing FPS counter
counter_fps = 0

# Define FPS Variable
FPS = 0

# Define tracker dictionary
tracker_dict = { 'csrt': cv2.TrackerCSRT_create,
                 'kcf' : cv2.TrackerKCF_create,
                 'boosting' : cv2.legacy.TrackerMOSSE_create(),
                 'mil': cv2.TrackerMIL_create,
                 'tld': cv2.legacy.TrackerTLD_create(),
                 'medianflow': cv2.legacy.TrackerMedianFlow_create(),
                 'mosse':cv2.legacy.TrackerMOSSE_create()}

# Initialize our tracker after the object
tracker = tracker_dict['csrt']()

########################################################################################################################

# Function create 3 frames from the frame we read
def preproccess_frames(frame, last_frame):
    
    # Resize the main frame to (WIDTH, HEIGH) shape
    frame = cv2.resize(frame, (WIDTH, HEIGH))
        
    # Copy frame to work with deffrent variable
    frame_rgb = cv2.resize(frame, (int(WIDTH/SCALE), int(HEIGH/SCALE)))
    
    # Return mask frame 
    frame_mask = mask(frame_rgb, last_frame)
    
    # Return mask for tracking
    tracking_mask = mask_tracking(frame_rgb, last_frame)
    
    # Define last frame
    last_frame = frame_rgb.copy()
    
    return frame, frame_rgb, tracking_mask, frame_mask, last_frame

# Function return 3-Dimension frame
def expands_dimensions(frame):
    
    new_image = np.zeros((frame.shape[0], frame.shape[1], 3), np.uint8)
    new_image[:, :, 0] = frame
    new_image[:, :, 1] = frame
    new_image[:, :, 2] = frame
    
    return new_image

# Convert frame from rgb to gray
def gray_frame(frame_rgb):
    
    # Converting captured frame to GRAY by OpenCV function    
    gray_frame = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2GRAY)
    frame_gray = np.zeros(frame_rgb.shape, np.uint8)
    frame_gray[:,:,0] = gray_frame
    frame_gray[:,:,1] = gray_frame
    frame_gray[:,:,2] = gray_frame
    
    return frame_gray

# Function define the mask
def mask(frame_rgb, background):
    
    # Converting captured frame to GRAY by OpenCV function    
    frame_gray = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2GRAY)
    
    # Create one more frame with Gaussian blur
    frame_gray = cv2.GaussianBlur(frame_gray, (25, 25), 0)  

    # Converting captured frame to GRAY by OpenCV function        
    background = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)
    
    # Create one more frame with Gaussian blur
    background = cv2.GaussianBlur(background, (25, 25), 0)    
    
    # Return mask to detect change between two frames   
    abs_diff = cv2.absdiff(frame_gray, background)
    
    # Function exclude values that ara more than treshhold = 15 0 and more than 255
    _, mask = cv2.threshold(abs_diff, 20, 255, cv2.THRESH_BINARY)
    
    # Dilates the object in the frame
    dilated_mask = cv2.dilate(mask, None, iterations = 5) 
    
    return dilated_mask

def mask_tracking(frame_RGB, last_frame):
    
    # Converting captured frame to GRAY by OpenCV function    
    frame_gray = cv2.cvtColor(frame_RGB, cv2.COLOR_RGB2GRAY)
    
    # Converting captured frame to GRAY by OpenCV function        
    last_frame = cv2.cvtColor(last_frame, cv2.COLOR_BGR2GRAY)
     
    # Return mask to detect change between two frames   
    abs_diff = cv2.absdiff(frame_gray, last_frame)
    
    # Function exclude values that ara more than treshhold = 15 0 and more than 255
    _, abs_diff_mask = cv2.threshold(abs_diff, DIFF_TRESH_HOLD, 255, cv2.THRESH_BINARY)

    # Expend mask dimension to 3 dimension
    mask_frame = expands_dimensions(abs_diff_mask)        
    
    return mask_frame

# Function display the frames on the screen in one window
def display_windows(frame, tracking_mask, frame_mask):  
        
    # Copy frame to work with deffrent variable
    tracking_mask = cv2.resize(tracking_mask, (WIDTH, HEIGH))
        
    # Copy frame to work with deffrent variable
    frame_mask = cv2.resize(frame_mask, (WIDTH, HEIGH))
        
    # Expend mask dimension to 3 dimension
    frame_mask = expands_dimensions(frame_mask)       
    
    # Create left window    
    main_window = np.hstack((frame, tracking_mask, frame_mask))

    # Plotting all the frames in one window
    cv2.imshow("Main_Window", main_window)     
    
# Function manage the frames reader variables like fps etc'
def reader_manger(FPS, fps_start, counter_fps, tracking_on):
       
    # Variable says if keep reading frame or quit
    quit = False

    # Stopping the timer for FPS
    fps_stop = timer()

    # Print FPS every 1 second
    if 1.0 <= fps_stop - fps_start:

        # Define FPS
        FPS = counter_fps

        # Reset FPS counter
        counter_fps = 0

        # Restart timer for FPS
        fps_start = timer()       

    # Function waits for key to be pressed    
    key = cv2.waitKey(15) % 256

    # If 'n' is pressed, we catchs the frame and define it as the background
    if key == ord('n'):
        tracking_on = False

    # If 'q' key is pressed then quit from app
    if key == ord('q'):
        quit = True   

    return FPS, fps_start, counter_fps, tracking_on, quit

# Function manage the detection and return status and coordinates
def detection_manager(frame, frame_rgb, frame_mask, counter_frames_prediction, counter_birds_prediction):
    
    label = "label"
    
    # No object to follow after found
    tracking_on = False
        
    # Increase prediction counter
    counter_frames_prediction += 1    
    
    # Function return array of all contours we found
    contours, _ = cv2.findContours(frame_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Sorted the contours and define the larger first
    contours = sorted(contours, key=cv2.contourArea, reverse=True)
    
    # Scan the contours list
    for contour in contours:

        # Return square area of the given contour
        contour_area = cv2.contourArea(contour)

        # Find contours between MIN_OBJECT_AREA to MAX_OBJECT_AREA
        if contour_area < MAX_OBJECT_AREA:
            if MIN_OBJECT_AREA < contour_area:

                # Increase prediction counter
                counter_frames_prediction += 1                
                
                # Function return the cutted rgb fragment in model input_shape
                cut_fragment_rgb, rectangle = cut_fragment(frame, frame_rgb, contour)
                
                # Function predict the cutted fragment and return the result
                scores, label, prediction_timer, detection_time = prediction_manager(cut_fragment_rgb)    
                
                # Start traking after the detected bird
                if label == "bird": 
                    
                    # Increasing Birds prediction counter
                    counter_birds_prediction += 1

                    # Function add object to the tracker
                    detected_bird(frame, rectangle)
                                
                    # Function return a bgr frame with rectangle around the cut fragment
                    frame = drawing_rectangle(frame, rectangle, label)  

#                     # Save detection event in date-time format
#                     save_detection_event(frame_bgr, detection_time, label)

#                     # Function return a frame with all the labels scores     
#                     scores_frame = bar_chart(scores[0], labels)  
                    
                    # Update tracking status
                    tracking_on = True
                    break

                # End of if not detected birds        
                else: 
                    # Increasing fail predictions counter
                    counter_fail_predictions = counter_frames_prediction - counter_birds_prediction              
            else:
                # Contour is a sorted list so all the rest items irrelevant
                break
    
    return frame, frame_rgb, label, tracking_on, counter_frames_prediction, counter_birds_prediction

# Function cut the detected fragment and return it
def cut_fragment(frame, frame_rgb, contour):
                
    # Get an approximate rectangle coordinates
    (x_min, y_min, box_width, box_height) = cv2.boundingRect(contour)

    # bounding_boxes contain x1, y1, x2, y2, coordinates and not width and heigh
    (x_min, y_min, box_width, box_height) = x_min*SCALE, y_min*SCALE, box_width*SCALE, box_height*SCALE
    
    # Cutting detected fragment from BGR frame
    cut_fragment_rgb_frame = frame[y_min: y_min + box_height, x_min: x_min + box_width]
    
    # Resize the fragment to the right frame shape in the main window
    cut_fragment_rgb_frame = cv2.resize(cut_fragment_rgb_frame, (WIDTH, HEIGH))
    
    # Create rectangle object from the boundaries coordinates
    rectangle = np.array([x_min, y_min, box_width, box_height])

    return cut_fragment_rgb_frame, rectangle

def prediction_manager(cut_fragment_rgb):
                             
    # Current time of detection object
    detection_time = strftime("%d/%m/%Y %H:%M:%S")  

    # Measuring classification time
    start = timer()

    # Function return all scores of model predictions
    scores = prediction_model(cut_fragment_rgb)       

    # End of Measuring classification time
    end = timer()

    # Calculate the time that needed to predict the fragment
    prediction_timer = end - start
    
    # Finds the lables array index by the max score index of model prediction
    index = np.argmax(scores)

    # Define the label for the cut_fragment from labels array
    label = labels[index]    
    
    return scores, label, prediction_timer, detection_time

# Function predict model's output from the cutted fragment    
def prediction_model(cut_fragment_rgb):
        
    # Create a copy of the cut_fragment_bgr frame  
    fragment = cut_fragment_rgb.copy()
    
    # Resizing frame to the right shape of the model's input
    fragment = cv2.resize(fragment, (width, hight), interpolation=cv2.INTER_CUBIC)
    
    # Extending dimension from (height, width, channels) to (1, heigh, width, channels)
    fragment = fragment[np.newaxis, :, :, :]

    # Predict score from model
    scores = model.predict(fragment)

    return scores

# Function drawing rectangle around the predicted object
def drawing_rectangle(frame, rectangle, label):
    
    # Get an approximate rectangle coordinates
    (x_min, y_min, box_width, box_height) = [int(a) for a in rectangle]

    # Drawing bounding box on the current BGR frame
    cv2.rectangle(frame, (x_min, y_min), (x_min + box_width, y_min + box_height), (0, 255, 0), 3)

    # Putting text with label on the current BGR frame
    cv2.putText(frame, label, (x_min - 5, y_min - 25), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 2)

    return frame 

# Function initialize variables of bird detected
def detected_bird(frame, rectangle):
    
    # Initialize Deque points
    points.clear()

    # Makes a sound to alert about bird
    #winsound.PlaySound(soundPath, winsound.SND_FILENAME)

    # Add the detected object to the tracker 
    tracker.init(frame, rectangle)   
    
# Function manage the tracking and return the status
def tracking_manager(frame, tracker, label, frame_mask, counter_frames_reading):     

    # Set the default status
    tracking_on = False

    # Get the bounding box from the frame
    (success, contour_box) = tracker.update(frame)

    # Keep tracking after the object
    if success:

        # Get the coordinates of the rectangle around the object
        (x, y, w, h) = [int(a) for a in contour_box]

        # Check if coordinates is in the frame boundaries
        if 0 <= x and x+w <= WIDTH and 0 <= y and y+h <= HEIGH:   

            # Cut the fragment from the mask frame
            cut_fragment_mask = frame_mask[int(y/SCALE):int((y+h)/SCALE), int(x/SCALE):int((x+w)/SCALE)]

            # Checking if tracking is still running after the object or not
            if((counter_frames_reading%tracking_check == 0 and np.sum(cut_fragment_mask) == 0)):      

                # Set tracking status OFF
                tracking_on = False
            else:
                # Set tracking status ON
                tracking_on = True                       

                # Drawing bounding box on the current BGR frame        
                cv2.rectangle(frame, (x,y), (x+w,y+h), (100,255,0), 2)

                # Putting text with label on the current BGR frame
                cv2.putText(frame, label, (x - 5, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 2)    

    return frame, tracker, tracking_on

In [2]:
# Loop reading frame by frame and processing them
while True:
    
    # Increasing FPS counter
    counter_fps += 1
    
    # Capturing frames one-by-one from camera
    ret, frame = camera.read()

    # If the frame was not retrieved then we break the loop
    if not ret or frame is None:
        break
     
    # Increasing frames counter
    counter_frames_reading += 1

    # Function return 3 diffrent kind of frames
    frame, frame_rgb, tracking_mask, frame_mask, last_frame = preproccess_frames(frame, last_frame)
    
    # Start Proccessing state only if is not quite frames
    if 0 < np.sum(tracking_mask):

        # Start Tracking state
        if tracking_on == True:

            # Increase tracking counter
            counter_frames_tracking += 1
            
            # Function manage the the tracking part  
            frame, tracker, tracking_on = tracking_manager(frame, tracker, label, frame_mask, counter_frames_reading)

        # Start Detection state
        else:

            # Increase tracking counter
            counter_frames_processing += 1

            # Function find countors around the objects and return drawn frame
            frame, frame_rgb, label, tracking_on,counter_frames_prediction,counter_birds_prediction = detection_manager(frame,
                                                                                                      frame_rgb, frame_mask, 
                                                                                                      counter_frames_prediction, 
                                                                                                      counter_birds_prediction)
        

    
    # Display all frames in one window
    display_windows(frame, tracking_mask, frame_mask)

    # Function manage the frames reader variables
    FPS, fps_start, counter_fps, tracking_on, quit = reader_manger(FPS, fps_start, counter_fps, tracking_on)

    # If quit is true so we stop read frames
    if quit == True:
        break

print("counter_frames_prediction: ", counter_frames_prediction )
print("counter_birds_prediction: ", counter_birds_prediction )
print("counter_fails_prediction: ", counter_frames_prediction - counter_birds_prediction )
    
# Releasing camera
camera.release()

# Destroying all opened OpenCV windows
cv2.destroyAllWindows()      

np.sum(frame_mask):  30442410
np.sum(frame_mask):  87416550
np.sum(frame_mask):  818805
np.sum(frame_mask):  1323450
1/1 [==============================] - 1s 700ms/step
np.sum(frame_mask):  49215
np.sum(frame_mask):  253215
np.sum(frame_mask):  52530
np.sum(frame_mask):  276930
np.sum(frame_mask):  468690
np.sum(frame_mask):  832320
np.sum(frame_mask):  621690
np.sum(frame_mask):  1224765
np.sum(frame_mask):  554370
np.sum(frame_mask):  1175805
np.sum(frame_mask):  52275
np.sum(frame_mask):  267750
np.sum(frame_mask):  204255
np.sum(frame_mask):  833085
np.sum(frame_mask):  805545
np.sum(frame_mask):  1432845
1/1 [==============================] - 0s 110ms/step
np.sum(frame_mask):  905760
np.sum(frame_mask):  1507050
np.sum(frame_mask):  989655
np.sum(frame_mask):  1984410
np.sum(frame_mask):  950895
np.sum(frame_mask):  1897200
np.sum(frame_mask):  893775
np.sum(frame_mask):  1656990
np.sum(frame_mask):  965430
np.sum(frame_mask):  1450440
np.sum(frame_mask):  52275
np.sum(frame_mask

np.sum(frame_mask):  1008270
1/1 [==============================] - 0s 85ms/step
np.sum(frame_mask):  589050
np.sum(frame_mask):  1095480
1/1 [==============================] - 0s 85ms/step
np.sum(frame_mask):  957015
np.sum(frame_mask):  1649340
1/1 [==============================] - 0s 96ms/step
np.sum(frame_mask):  51765
np.sum(frame_mask):  244035
np.sum(frame_mask):  927180
np.sum(frame_mask):  1595790
1/1 [==============================] - 0s 88ms/step
np.sum(frame_mask):  51765
np.sum(frame_mask):  258570
np.sum(frame_mask):  962880
np.sum(frame_mask):  1698300
np.sum(frame_mask):  686460
np.sum(frame_mask):  1298205
np.sum(frame_mask):  878985
np.sum(frame_mask):  1471095
np.sum(frame_mask):  932790
np.sum(frame_mask):  1582785
np.sum(frame_mask):  51765
np.sum(frame_mask):  262395
np.sum(frame_mask):  121635
np.sum(frame_mask):  580635
np.sum(frame_mask):  94095
np.sum(frame_mask):  449055
np.sum(frame_mask):  51765
np.sum(frame_mask):  495720
np.sum(frame_mask):  825690
np.su

np.sum(frame_mask):  874140
np.sum(frame_mask):  1373175
np.sum(frame_mask):  48960
np.sum(frame_mask):  257040
np.sum(frame_mask):  48960
np.sum(frame_mask):  482715
np.sum(frame_mask):  51765
np.sum(frame_mask):  328950
np.sum(frame_mask):  48705
np.sum(frame_mask):  342720
np.sum(frame_mask):  371280
np.sum(frame_mask):  686970
1/1 [==============================] - 0s 95ms/step
np.sum(frame_mask):  48960
np.sum(frame_mask):  250920
np.sum(frame_mask):  233580
np.sum(frame_mask):  677790
np.sum(frame_mask):  51765
np.sum(frame_mask):  554625
np.sum(frame_mask):  102510
np.sum(frame_mask):  398565
np.sum(frame_mask):  52020
np.sum(frame_mask):  393975
np.sum(frame_mask):  48705
np.sum(frame_mask):  287640
np.sum(frame_mask):  48960
np.sum(frame_mask):  301410
np.sum(frame_mask):  48960
np.sum(frame_mask):  311355
np.sum(frame_mask):  111945
np.sum(frame_mask):  459765
np.sum(frame_mask):  669120
np.sum(frame_mask):  982260
np.sum(frame_mask):  51765
np.sum(frame_mask):  275400
np.sum